<a href="https://colab.research.google.com/github/Sohammhatre10/Predicto/blob/main/NewsAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.4 MB/s eta 0:00:00


In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at human-centered-summarization/financial-summarization-pegasus and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

In [ ]:
monitored_ticker = ['Bitcoin', 'Dogecoin', 'Litecoin', 'Ethereum', 'XRP']

In [ ]:
def search_for_crypto_news(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&sca_esv=554867176&rlz=1C1ONGR_enIN1063IN1063&tbs=sbd:1,qdr:d&tbm=nws&sxsrf=AB5stBjN5-syjYtVHcPU9P30_SJyTV9fUg:1691523802642&source=lnt&sa=X&ved=2ahUKEwi7s4qb6c2AAxUmpVYBHfCNBSoQpwV6BAgBEAg&biw=1280&bih=595&dpr=1.5".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs

In [ ]:
raw_urls = {}
for ticker in monitored_ticker:
  raw_urls[ticker] = search_for_crypto_news(ticker)

In [ ]:
import re

In [ ]:
exclude_list = ['maps', 'policies','preferences', 'accounts', 'support']

In [ ]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [ ]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_ticker}
cleaned_urls

{'Bitcoin': ['https://theglobalherald.com/business/finance/stocks-fall-as-moodys-bank-downgrade-adds-to-august-woes-stock-market-news-today-august-8-2023/',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BBitcoin%26sca_esv%3D554867176%26rlz%3D1C1ONGR_enIN1063IN1063%26tbs%3Dsbd:1,qdr:d%26tbm%3Dnws%26sxsrf%3DAB5stBjN5-syjYtVHcPU9P30_SJyTV9fUg:1691523802642%26source%3Dlnt%26sa%3DX%26ved%3D2ahUKEwi7s4qb6c2AAxUmpVYBHfCNBSoQpwV6BAgBEAg%26biw%3D1280%26bih%3D595%26dpr%3D1.5%26pccc%3D1',
  'https://finance.yahoo.com/news/surprising-reason-behind-recent-influx-133311471.html',
  'https://finance.yahoo.com/news/next-crypto-gem-tv-show-133713834.html',
  'https://finance.yahoo.com/news/top-market-crypto-tech-politics-134141230.html',
  'https://finance.yahoo.com/news/hanetf-co-ceo-says-crypto-122808131.html',
  'https://finance.yahoo.com/news/gen-z-leading-way-crypto-131200924.html',
  'https://finance.yahoo.com/news/first-mover-americas-bitcoin-flirts-121856461.html',
  'https://finance.ya

In [ ]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [ ]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_ticker}
articles

{'Bitcoin': ['Home » Business Yahoo Finance published this video item, entitled “Stocks fall as Moody’s bank downgrade adds to August woes: Stock market news today | August 8, 2023” – below is their description. #stocks  #inflation #YahooFinance #recession #bitcoin #Biden #Stockmarket #coronavirus #memestocks #Fed #YahooFinance #investing #stockmarket #crypto  To get the latest market news check out finance.yahoo.com Stocks came under pressure Tuesday but recovered from steeper early after a warning about US bank health and surprisingly weak Chinese trade data sent shivers through markets. The Dow Jones Industrial Average (^DJI) dropped about 0.4%, or 150 points. Meanwhile, the S&P 500 (^GSPC) was down about 0.4%, and the tech-heavy Nasdaq Composite (^IXIC) slipped 0.8%, both paring losses of more than 1% earlier Tuesday. Worries about US banks were revived after Moody’s downgraded 10 small and midsize institutions, which came with a warning it could lower credit ratings for some of th

In [ ]:
btc = articles["Bitcoin"]
eth = articles["Ethereum"]
lte = articles["Litecoin"]
xrp = articles["XRP"]
doge = articles["Dogecoin"]

In [ ]:
def summarize(articles):
    summaries = []
    for ticker, article_list in articles.items():
        article_text = '\n\n'.join(article_list)  # Join the list of articles into a single string
        ticker_summaries = []
        for chunk in article_text.split('\n\n'):  # Split the article into paragraphs (you can adjust this based on your article structure)
            input_ids = tokenizer.encode(chunk, return_tensors='pt', max_length=512, truncation=True)
            output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
            summary = tokenizer.decode(output[0], skip_special_tokens=True)
            ticker_summaries.append(summary)
        summaries.append({ticker: ticker_summaries})
    return summaries

# Assuming you have a dictionary with lists of articles for each ticker (e.g., btc, eth, doge, xrp, lte)
articles = {
    'Bitcoin': btc,
    'Ethereum': eth,
    'Dogecoin': doge,
    'XRP': xrp,
    'Litecoin': lte
}

# Call the summarize function with the articles dictionary
summaries = summarize(articles)
print(summaries)

[{'Bitcoin': ['Moody’s cuts 10 small and midsize banks to A2 from Aa3', 'Follow us on Instagram, Snapchat and Twitter for the latest news.', '.', 'All images are copyrighted.', 'Firms re-file for Ether futures ETF, hoping for approval. SEC ‘readiness’ for Ether ETF previously indicated', '‘The Next Crypto Gem’ will pit 16 contestants against each other. Executive producer says show aims to bring mainstream adoption', "Donald Trump mocks Chris Christie's weight at New Hampshire rally. Apple CEO: End-To-End Encryption on iPhone will have big impact", 'HANetf co-founder and co-CEO Hector McNeil speaks with Thomas Warner.', 'The data reveals that Generation Z users account for 44% of all copy trading users. Over 109,000 followers made 74 million USDT in cumulative profits in first half of 2023', 'Fed starts new program for overseeing banks’ crypto activity. El Salvador’s Crypto Exchange seeks to build legitimacy worldwide', 'Russia to pilot digital rouble with consumers on Aug. 15. Central

In [ ]:
from transformers import pipeline
import pandas as pd


# Initialize the sentiment analysis pipeline
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model_revision = "af0f99b"
sentiment_classifier = pipeline('sentiment-analysis', model=model_name, revision=model_revision)

def analyze_sentiment(summaries):
    sentiment_results = sentiment_classifier(summaries)  # Use the pipeline directly on the list of summaries
    return sentiment_results

# Assuming you want to analyze the sentiment for the 'BTC' ticker's summaries
btc_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('Bitcoin', [])]
eth_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('Ethereum', [])]  # List of summaries for ETH
doge_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('Dogecoin', [])]  # List of summaries for DOGE
xrp_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('XRP', [])]  # List of summaries for XRP
lte_summaries = [summary for ticker_summaries in summaries for summary in ticker_summaries.get('Litecoin', [])]  # List of summaries for LTE

sentiment_results_btc = analyze_sentiment(btc_summaries)
sentiment_results_eth = analyze_sentiment(eth_summaries)
sentiment_results_doge = analyze_sentiment(doge_summaries)
sentiment_results_xrp = analyze_sentiment(xrp_summaries)
sentiment_results_lte = analyze_sentiment(lte_summaries)


data = {
    'Ticker': ['Bitcoin'] * len(sentiment_results_btc) + ['Ethereum'] * len(sentiment_results_eth) +
              ['Dogecoin'] * len(sentiment_results_doge) + ['XRP'] * len(sentiment_results_xrp) +
              ['Litecoin'] * len(sentiment_results_lte),
    'Result_state': [result['label'] for result in sentiment_results_btc] +
                       [result['label'] for result in sentiment_results_eth] +
                       [result['label'] for result in sentiment_results_doge] +
                       [result['label'] for result in sentiment_results_xrp] +
                       [result['label'] for result in sentiment_results_lte],
    'Score': [result['score'] for result in sentiment_results_btc] +
                       [result['score'] for result in sentiment_results_eth] +
                       [result['score'] for result in sentiment_results_doge] +
                       [result['score'] for result in sentiment_results_xrp] +
                       [result['score'] for result in sentiment_results_lte],
}

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('sentiment_analysis_results.csv', index=False)

In [ ]:
import math

coin_results = {}
coins = ['Bitcoin', 'Ethereum', 'XRP', 'Litecoin', 'Dogecoin']

for coin in coins:
    positive_sum = df[(df['Ticker'] == coin) & (df['Result_state'] == 'POSITIVE')]['Score'].sum()
    negative_sum = df[(df['Ticker'] == coin) & (df['Result_state'] == 'NEGATIVE')]['Score'].sum()
    coin_results[coin] = 1-(((positive_sum - negative_sum)/(pow(len(df[(df['Ticker'] == coin)]),(13589029/10000000)))))
print(coin_results)





{'Bitcoin': 1.1294038115036744, 'Ethereum': 1.1157701807789262, 'XRP': 1.1452698152775214, 'Litecoin': 0.9957941890006753, 'Dogecoin': 1.0011816075076794}
